In [1]:
import numpy as np
import pandas as pd

In [2]:
pd.set_option('display.max_rows', None)  # 显示所有行
pd.set_option('display.max_columns', None)  # 显示所有列
pd.set_option('expand_frame_repr', False)  # 即“禁止换行”
# pd.set_option('display.precision', 2) #展示两位小数点
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [3]:
df_data_交易=pd.read_csv('data/账户交易信息.csv')
df_data_交易.columns = ['交易流水序号','账户代号','对方账号','借贷标志','交易金额','交易余额','对方行号','交易日期','交易时间','交易渠道','摘要代号','对方名称长度']
df_data_交易 = df_data_交易.sort_values(["交易日期" , "交易时间"])
df_data_账户=pd.read_csv('data/账户静态信息.csv')
df_data_账户.columns = ['账户代号','开户日期','开户行代号','客户性别','年龄']
df_label_train=pd.read_csv('data/训练集标签.csv')
df_label_train.columns = ['账户代号','label']
df_label_test=pd.read_csv('data/test_dataset.csv')
df_label_test.columns = ['账户代号']## 读取数据
df_label_all=pd.concat([df_label_train,df_label_test])

In [5]:
list_交易_金额为负_index = list(df_data_交易[df_data_交易["交易金额"] < 0].index)
list_mz_index = [] ;
list_交易_金额为负_对应的失败交易_index = [] ;
for i in list_交易_金额为负_index :
    S_now = df_data_交易.loc[i] ;
    j = i ;
    while (True) :
        j -= 1 ;
        S_nex = df_data_交易.loc[j] ;
        if (S_now["交易金额"] == -S_nex["交易金额"]) :
            if (S_now["账户代号"] == S_nex["账户代号"]) :
                list_交易_金额为负_对应的失败交易_index.append(j) ;
                list_mz_index.append(i) ;
                break ;
list_交易_抹账需要去除_index = list_交易_金额为负_index + list_交易_金额为负_对应的失败交易_index ;
df_data_交易_去除抹账后=df_data_交易[~df_data_交易.index.isin(list_交易_抹账需要去除_index)]

In [6]:
df_data_交易_去除抹账后.shape

(812264, 12)

In [7]:
df_data_交易.shape

(816270, 12)

In [10]:
816270 - 812264

4006

In [22]:
clicode = "2DB552D3B7D0FAAD"
df1 = df_data_交易_去除抹账后[df_data_交易_去除抹账后["账户代号"] == clicode].sort_values(by=["交易日期","交易时间"])

In [ ]:
df1

In [ ]:
df1[df1["借贷标志"] == 0].groupby("对方账号")["交易金额" , "交易余额"].agg(["mean" , "max" , "min", "count"]).sort_values(by=[("交易金额" , "count")] , ascending=False)
# df1
